In [90]:
from typing import Tuple, NamedTuple, FrozenSet
from collections import defaultdict
from math import inf
from heapq import heappop, heappush
from functools import cache
from itertools import combinations

import black
import jupyter_black

jupyter_black.load(lab=True, target_version=black.TargetVersion.PY310)

In [91]:

# Day 11: Radioisotope Thermoelectric Generators
class State(NamedTuple):
    "Describes current elevator level, microchips and generators per floor."
    elevator: int
    floors: Tuple[FrozenSet, FrozenSet, FrozenSet, FrozenSet]


def fs(*items):
    return frozenset(sorted(items))


def combos(items):
    "1 and 2 long combinations of `items`."
    for i in (1, 2):
        for x in combinations(items, i):
            yield fs(*x)



def legal_floor(floor):
    """
    The chips are prototypes and don't have normal radiation shielding, but they do have the ability to generate an electromagnetic radiation shield when powered. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.
    """
    generators = {gen[0] for gen in floor if gen[-1] == "G"}
    microchips = {chip[0] for chip in floor if chip[-1] == "M"}
    if not generators:
        # A floor without generators is safe
        return True
    return all((chip in generators) for chip in microchips)

def moves(state: State):
    legal_floors = {0, 1, 2, 3}
    L, floors = state
    for L2 in {L + 1, L - 1} & legal_floors:
        # bring one or two things to the new floors
        for bring in combos(floors[L]):
            new_floors = list(range(len(legal_floors)))
            for floor in legal_floors:
                if floor == L2:
                    new_floors[floor] = bring | floors[L2]
                elif floor == L:
                    new_floors[floor] = floors[floor] - bring
                else:
                    new_floors[floor] = floors[floor]
            if legal_floor(new_floors[L]) and legal_floor(new_floors[L2]):
                yield State(L2, tuple(new_floors))


def done(state: State) -> bool:
    return state.elevator == 3 and not any(state.floors[i] for i in range(3))


def a_star(state: State):
    # A star with defaultdict and native heapq instead of PriorityQueue
    def heuristic(state) -> int:
        "An estimate of the number of moves needed to move everything to top."
        total = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
        return total // 2  # Can move two items in one move.

    frontier = []
    heappush(frontier, (0, state))
    came_from = {state: None}
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[state] = 0

    while frontier:
        current = heappop(frontier)[1]

        if done(current):
            return cost_so_far[current]

        for next in moves(current):
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next)
                heappush(frontier, (priority, next))
                came_from[next] = current


Ø = frozenset()

example = State(0, (fs("HM", "LM"), fs("HG"), fs("LG"), Ø))  # 11

part1 = State(
    0, (fs("PM", "PG"), fs("CG", "cG", "RG", "pG"), fs("CM", "cM", "RM", "pM"), Ø)
)  # 33

part2 = State(
    0,
    (
        fs("EG", "EM", "DG", "DM", "PM", "PG"),
        fs("CG", "cG", "RG", "pG"),
        fs("CM", "cM", "RM", "pM"),
        Ø,
    ),
)  # 57

%time print("Part 1:", a_star(part1))

Part 1: 33
CPU times: user 8.74 s, sys: 51.8 ms, total: 8.79 s
Wall time: 8.84 s


In [104]:
# Cache some functions
# Day 11: Radioisotope Thermoelectric Generators
class State(NamedTuple):
    "Describes current elevator level, microchips and generators per floor."
    elevator: int
    floors: Tuple[FrozenSet, FrozenSet, FrozenSet, FrozenSet]


@cache  # No significant difference
def fs(*items):
    return frozenset(sorted(items))


@cache  # from 5.5s -> 4.7s
def combos(items):
    "1 and 2 long combinations of `items`."
    return [fs(*x) for i in (1, 2) for x in combinations(items, i)]


@cache  # from 9.5s -> 5.5s
def legal_floor(floor):
    """
    The chips are prototypes and don't have normal radiation shielding, but they do have the ability to generate an electromagnetic radiation shield when powered. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.
    """
    generators = {gen[0] for gen in floor if gen[-1] == "G"}
    microchips = {chip[0] for chip in floor if chip[-1] == "M"}
    if not generators:
        # A floor without generators is safe
        return True
    return all((chip in generators) for chip in microchips)


def moves(state: State):
    legal_floors = {0, 1, 2, 3}
    L, floors = state
    for L2 in {L + 1, L - 1} & legal_floors:
        # bring one or two things to the new floors
        for bring in combos(floors[L]):
            new_floors = [0] * 4  # 4.7s -> 4s
            for floor in legal_floors:
                if floor == L2:
                    new_floors[floor] = bring | floors[L2]
                elif floor == L:
                    new_floors[floor] = floors[floor] - bring
                else:
                    new_floors[floor] = floors[floor]
            if legal_floor(new_floors[L]) and legal_floor(new_floors[L2]):
                yield State(L2, tuple(new_floors))


def done(state: State) -> bool:
    return state.elevator == 3 and not any(state.floors[i] for i in range(3))


def a_star(state: State):
    # A star with defaultdict and native heapq instead of PriorityQueue
    def heuristic(state) -> int:
        "An estimate of the number of moves needed to move everything to top."
        total = sum(len(floor) * i for (i, floor) in enumerate(reversed(state.floors)))
        return total // 2  # Can move two items in one move.

    frontier = []
    heappush(frontier, (0, state))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[state] = 0

    while frontier:
        current = heappop(frontier)[1]

        if done(current):
            print(len(cost_so_far), "states explored.")
            return cost_so_far[current]

        for next in moves(current):
            new_cost = cost_so_far[current] + 1
            if new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next)
                heappush(frontier, (priority, next))


Ø = frozenset()

example = State(0, (fs("HM", "LM"), fs("HG"), fs("LG"), Ø))  # 11

part1 = State(
    0, (fs("PM", "PG"), fs("CG", "cG", "RG", "pG"), fs("CM", "cM", "RM", "pM"), Ø)
)  # 33

part2 = State(
    0,
    (
        fs("EG", "EM", "DG", "DM", "PM", "PG"),
        fs("CG", "cG", "RG", "pG"),
        fs("CM", "cM", "RM", "pM"),
        Ø,
    ),
)  # 57

%time print("Part 1:", a_star(part1))

156510 states explored.
Part 1: 33
CPU times: user 4.23 s, sys: 27.7 ms, total: 4.26 s
Wall time: 4.28 s


In [99]:
# Skip the State class and heuristics function (i.e. Dijkstra instead of A*). About the
# same result. The function helps reduce search space, but if it is too expensive to run
# the end result might not be better. This version shows no significant difference, but
# less code is easier understand and debug. The State class should probably still be kept,
# it makes it slightly easier to understand.

# If using heuristic 156,510 states are explored. Without it 158,221 states are explored.


# Day 11: Radioisotope Thermoelectric Generators
@cache  # No significant difference
def fs(*items):
    return frozenset(sorted(items))


@cache  # from 5.5s -> 4.7s
def combos(items):
    "1 and 2 long combinations of `items`."
    return [fs(*x) for i in (1, 2) for x in combinations(items, i)]


@cache  # from 9.5s -> 5.5s
def legal_floor(floor):
    """
    The chips are prototypes and don't have normal radiation shielding, but they do have the ability to generate an electromagnetic radiation shield when powered. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.
    """
    generators = {gen[0] for gen in floor if gen[-1] == "G"}
    microchips = {chip[0] for chip in floor if chip[-1] == "M"}
    if not generators:
        # A floor without generators is safe
        return True
    return all((chip in generators) for chip in microchips)


def moves(L, floors):
    legal_floors = {0, 1, 2, 3}
    for L2 in {L + 1, L - 1} & legal_floors:
        # bring one or two things to the new floors
        for bring in combos(floors[L]):
            new_floors = [0] * 4  # 4.7s -> 4s
            for floor in legal_floors:
                if floor == L2:
                    new_floors[floor] = bring | floors[L2]
                elif floor == L:
                    new_floors[floor] = floors[floor] - bring
                else:
                    new_floors[floor] = floors[floor]
            if legal_floor(new_floors[L]) and legal_floor(new_floors[L2]):
                yield L2, tuple(new_floors)


def done(L, floors) -> bool:
    return L == 3 and not any(floors[i] for i in range(3))


def a_star(L, floors):
    # A star with defaultdict and native heapq instead of PriorityQueue
    frontier = []
    heappush(frontier, (0, (L, floors)))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[(L, floors)] = 0

    while frontier:
        L, floors = heappop(frontier)[1]

        if done(L, floors):
            return cost_so_far[(L, floors)]

        for next_L, next_floors in moves(L, floors):
            new_cost = cost_so_far[(L, floors)] + 1
            if new_cost < cost_so_far[(next_L, next_floors)]:
                cost_so_far[(next_L, next_floors)] = new_cost
                heappush(frontier, (new_cost, (next_L, next_floors)))


part1 = (
    fs("PM", "PG"),
    fs("CG", "cG", "RG", "pG"),
    fs("CM", "cM", "RM", "pM"),
    frozenset(),
)  # 33

%time print("Part 1:", a_star(0, part1))

158221
Part 1: 33
CPU times: user 4.32 s, sys: 34.9 ms, total: 4.35 s
Wall time: 4.38 s


In [94]:
# Use positive ints for generators and negative for microchips instead of strings - worse performance

# Day 11: Radioisotope Thermoelectric Generators
# @cache # Gives *worse* performance
def fs(*items):
    return frozenset(sorted(items))


@cache  # from 5.5s -> 4.7s
def combos(items):
    "1 and 2 long combinations of `items`."
    return [fs(*x) for i in (1, 2) for x in combinations(items, i)]


@cache  # from 9.5s -> 5.5s
def legal_floor(floor):
    """
    The chips are prototypes and don't have normal radiation shielding, but they do have the ability to generate an electromagnetic radiation shield when powered. Unfortunately, they can only be powered by their corresponding RTG. An RTG powering a microchip is still dangerous to other microchips.
    """
    generators = {gen for gen in floor if gen > 0}
    microchips = {-chip for chip in floor if chip < 0}
    if not generators:
        # A floor without generators is safe
        return True
    return all((chip in generators) for chip in microchips)


def moves(L, floors):
    legal_floors = {0, 1, 2, 3}
    for L2 in {L + 1, L - 1} & legal_floors:
        # bring one or two things to the new floors
        for bring in combos(floors[L]):
            new_floors = [0] * 4  # 4.7s -> 4s
            for floor in legal_floors:
                if floor == L2:
                    new_floors[floor] = bring | floors[L2]
                elif floor == L:
                    new_floors[floor] = floors[floor] - bring
                else:
                    new_floors[floor] = floors[floor]
            if legal_floor(new_floors[L]) and legal_floor(new_floors[L2]):
                yield L2, tuple(new_floors)


def done(L, floors) -> bool:
    return L == 3 and not any(floors[i] for i in range(3))


def a_star(L, floors):
    # A star with defaultdict and native heapq instead of PriorityQueue
    frontier = []
    heappush(frontier, (0, (L, floors)))
    cost_so_far = defaultdict(lambda: inf)
    cost_so_far[(L, floors)] = 0

    while frontier:
        L, floors = heappop(frontier)[1]

        if done(L, floors):
            return cost_so_far[(L, floors)]

        for next_L, next_floors in moves(L, floors):
            new_cost = cost_so_far[(L, floors)] + 1
            if new_cost < cost_so_far[(next_L, next_floors)]:
                cost_so_far[(next_L, next_floors)] = new_cost
                heappush(frontier, (new_cost, (next_L, next_floors)))


Ø = frozenset()

# Use positive ints for generators and negative for microchips instead of strings
part1 = (
    fs(-1, 1),
    fs(2, 3, 4, 5),
    fs(-2, -3, -4, -5),
    Ø,
)  # 33

%time print("Part 1:", a_star(0, part1))

Part 1: 33
CPU times: user 4.97 s, sys: 40.5 ms, total: 5.01 s
Wall time: 5.07 s
